In [1]:
#!pip install deepface

In [2]:
import os
import sys
from deepface import DeepFace
from PIL import Image, ImageDraw
from IPython.display import display
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
sys.path.append('aux_scripts.py')

2025-01-09 20:06:46.773612: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 20:06:46.825834: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-09 20:06:46.868948: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736449606.914318   80860 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736449606.924478   80860 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-09 20:06:46.969590: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Define the folder and image name

In [3]:
folder_name = "incorrect_mask/"
modified_folder_name = "incorrect_mask/mod/"
cropped_folder_name = "incorrect_mask/cropped/"

# Create the processed and cropped folders if they don't exist
os.makedirs(modified_folder_name, exist_ok=True)
os.makedirs(cropped_folder_name, exist_ok=True)

for image_name in os.listdir(folder_name):
    # Only process if the file is an image (e.g., PNG, JPG)
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Load image
        image_path = os.path.join(folder_name, image_name)
        image = Image.open(image_path)
        image_draw = Image.open(image_path)

        # Analyze the image
        analysis = DeepFace.analyze(img_path=image_path, detector_backend='retinaface', enforce_detection=False)
        
        # Draw bounding boxes for each face detected
        draw = ImageDraw.Draw(image_draw)
        for idx, item in enumerate(analysis):
            region = item['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            offset_x = 0.2
            offset_y = 0.15

            # Crop the detected face
            cropped_face = image.crop((x-((w*offset_x)/2), y-((h*offset_y)/2), x + w + w*offset_x, y + h + h*offset_y))
            cropped_face_path = os.path.join(cropped_folder_name, f"{image_name}_face_{idx + 1}.png")
            cropped_face.save(cropped_face_path)
            #print(f"Cropped face saved as {cropped_face_path}")

            # Draw rectangle on the original image
            width_line = 2
            draw.rectangle([(x-((w*offset_x)/2), y-((h*offset_y)/2)), (x + w + w*offset_x, y + h + h*offset_y)], outline='green', width=width_line)

        # Save the image with bounding boxes
        modified_image_path = os.path.join(modified_folder_name, "MOD_" + image_name)
        image_draw.save(modified_image_path)
        
        # Optionally, display the processed image
        #modified_image = Image.open(modified_image_path)
        #display(modified_image)


W0000 00:00:1736449609.895695   80860 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2025-01-09 20:06:52.871922: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18923520 exceeds 10% of free system memory.
2025-01-09 20:06:54.050855: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18923520 exceeds 10% of free system memory.
2025-01-09 20:06:54.061938: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 19108560 exceeds 10% of free system memory.
2025-01-09 20:06:54.067420: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 100925440 exceeds 10% of free system memory.
2025-01-09 20:06:54.119648: 